##### Imports

In [1]:
import sys
from pathlib import Path
import warnings

import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import sys
# Custom library paths
sys.path.extend(['../', './scr'])

from scr.utils import set_seed
from scr.utils import read_words

set_seed(42)

import torch
import torch.nn as nn

torch.set_float32_matmul_precision('medium')

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##### Feature Engineering

##### Data Reading and Feature Engineering

In [2]:
from scr.feature_engineering import build_feature_set, \
    process_single_word_inference
from scr.utils import *

import random

# MASK_PROB = 0.5

# Limit the number of words to a smaller number for debugging
word_list = read_words('/home/sayem/Desktop/Hangman/words_250000_train.txt', limit=None)
# word_list = word_list[:100]
# # # Randomly select 1000 words
# # unseen_words = random.sample(word_list, 1000)c

In [3]:
import torch
from scr.feature_engineering import add_features_for_training, calculate_char_frequencies
import random
# # Initialize lists for features, labels, missed characters, and original words
# all_features, all_labels, all_missed_chars, original_words = [], [], [], []

char_frequency = calculate_char_frequencies(word_list)
max_word_length = max(len(word) for word in word_list)
MASK_PROB = 0.8
NGRAM_N = 3

feature_set, label, missed_chars = add_features_for_training(
        word_list[0], char_frequency, max_word_length, MASK_PROB, NGRAM_N)

features, labels, missed_chars_list, original_words = [], [], [], []

for word in word_list:
    # Process each word to get its features, label, and missed characters
    feature_set, label, missed_chars = add_features_for_training(
        word, char_frequency, max_word_length, MASK_PROB, NGRAM_N
    )

    # Add features and labels to the lists without squeezing
    features.append(feature_set)
    labels.append(torch.tensor(label, dtype=torch.float))
    missed_chars_list.append(missed_chars)
    original_words.append(word)  # Store the original word

# # Convert lists to tensors
# all_features_tensor = [features.squeeze(0) for features in all_features]  # Remove batch dimension
# labels_tensor = [label.squeeze(0) for label in all_labels]  # Remove batch dimension
# missed_chars_tensor = [missed_chars.squeeze(0) for missed_chars in all_missed_chars]  # Remove batch dimension

In [4]:
from scr.feature_engineering import process_single_word_inference

def process_inference_word(word, char_frequency, max_word_length, ngram_n=2):
    feature_set, missed_chars = process_single_word_inference(word, \
        char_frequency, max_word_length, ngram_n=ngram_n)
    return feature_set.squeeze(0), missed_chars.squeeze(0)  # Remove batch dimension

# Example usage
inference_word = "_p_"
inference_features, inference_missed_chars = \
    process_inference_word(inference_word, char_frequency, max_word_length, ngram_n=NGRAM_N)

In [5]:
inference_features

tensor([[ 0.0000,  0.1034,  0.0000,  0.0000,  0.0000],
        [16.0000,  0.1034,  0.0345,  0.2814, 16.0000],
        [ 0.0000,  0.1034,  0.0690,  0.0000,  0.0000]])

In [6]:
inference_missed_chars.shape

torch.Size([27])

In [7]:
len(word_list)

227300

##### Dataset Building

In [8]:
from scr.dataset import HangmanDataset, collate_fn
from torch.utils.data import DataLoader, Dataset

dataset = HangmanDataset(features, \
    labels, missed_chars_list, original_words)


data_loader = DataLoader(dataset, batch_size=32, \
    shuffle=True, collate_fn=collate_fn)

# Example of iterating over the DataLoader
i = 0
for i, batch in enumerate(data_loader):
    inputs, labels, miss_chars, lengths, original_words = batch
    i += 1

    # print(inputs.shape)
    # print(labels.shape)
    # print(missed_chars.shape)
    # print(lengths.shape)
    # print()

len(dataset)

227300

##### Model Building

In [9]:
# In your main script or Jupyter Notebook
from scr.model import RNN
from scr.feature_engineering import process_single_word_inference, \
    char_to_idx, idx_to_char, calculate_char_frequencies, \
        get_missed_characters
from scr.game import simulate_game, \
    predict_next_character

# Configuration for the RNN model
# Configuration for the RNN model
config = {
    'rnn': 'LSTM',
    'vocab_size': 27,  # Assuming 26 letters + 1 for underscore
    'hidden_dim': 256,
    'num_layers': 2,
    'embedding_dim': 200,
    'output_mid_features': 100,
    'miss_linear_dim': 100,
    'dropout': 0.7,
    'use_embedding': True,
    'lr': 0.00001,
    'input_feature_size': 5 # Number of features excluding the embedding dimension
}

# Initialize RNN model
model = RNN(config)
model = model.to(device)

model.save_model('models/model.pth') 

# Prepare your dataset, train the model, etc.

# # Example of using predict_next_character in a game scenario
# word = "apple"

In [10]:
current_masked_word = "_ppl_"

# missed_chars = get_missed_characters(word, char_to_idx)

predicted_char = predict_next_character(model, \
    current_masked_word, \
    char_frequency, max_word_length, \
        device=device, use_initial_guess=True)

# predicted_index = predict_next_character_beam_search(model, current_masked_word, \
#     missed_chars, \
#     char_frequency, max_word_length, \
#     device, normalize=True, beam_width=3)
        
# predicted_char = idx_to_char[predicted_index]

predicted_char

'i'

In [11]:
# %%capture

# from scr.game import predict_next_character, simulate_game
# from scr.model import RNN
# import random
# # random.seed(400)
# # Your existing code for initializing the model, etc.

# import random

# def play_multiple_games(model, num_games, word_list, \
#     char_to_idx, idx_to_char, char_frequency, max_word_length, device):
#     game_results = []
#     sampled_words = random.sample(word_list, num_games)  # Select unique words

#     for random_word in sampled_words:
#         with torch.no_grad():
#             won, final_word, attempts_used = simulate_game(
#                 model, 
#                 random_word, 
#                 char_to_idx, 
#                 idx_to_char, 
#                 char_frequency, 
#                 max_word_length, 
#                 device, 
#                 normalize=True, 
#                 max_attempts=6
#             )
#         game_results.append((won, final_word, attempts_used))

#     return game_results

# # Example usage
# num_games = 10**4
# results = play_multiple_games(model, num_games, \
#     word_list, char_to_idx, idx_to_char, \
#         char_frequency, max_word_length, device)

# # Analyzing results
# total_wins = sum(result[0] for result in results)
# win_rate = (total_wins / num_games) * 100


##### Train

In [12]:
from torch.nn.utils.rnn import pad_sequence
import torch
from torch.utils.data import DataLoader

def train_one_epoch(model, data_loader, optimizer, device=device):
    total_actual_penalty = 0
    total_miss_penalty = 0
    total_batches = 0

    model.train()
    model.to(device)

    for i, batch in enumerate(data_loader):
        inputs, labels, miss_chars, lengths, original_words = batch

        inputs = inputs.to(device)
        labels = labels.to(device)
        miss_chars = miss_chars.to(device)
        lengths = lengths # .to(device)

        # print(f"Batch {i}: Inputs Shape: {inputs.shape}, Labels Shape: \
        # {labels.shape}, Lengths: {lengths.shape}, Miss Chars: {miss_chars.shape}")

        # # Run the model
        outputs = model(inputs, lengths, miss_chars)
        # print(f'NN output: {outputs.shape}')

       
        # # Calculate the custom loss
        actual_penalty, miss_penalty = model.calculate_loss(outputs, \
            labels, lengths, miss_chars, vocab_size=27, use_cuda=True)

        # # print(actual_penalty)
        # # print(miss_penalty)

        total_actual_penalty += actual_penalty.item()
        total_miss_penalty += miss_penalty.item()
        total_batches += 1

        optimizer.zero_grad()
        actual_penalty.backward()  # Backpropagation for the actual_penalty
        optimizer.step()
        # break

    avg_actual_penalty = total_actual_penalty / total_batches if total_batches > 0 else 0
    avg_miss_penalty = total_miss_penalty / total_batches if total_batches > 0 else 0
    return avg_actual_penalty, avg_miss_penalty


import random
import collections

def validate_one_epoch(model, val_loader, char_to_idx, idx_to_char, \
    char_frequency, max_word_length, device=device, max_games_per_epoch=10 ** 4):
    model.eval()
    total_wins = 0
    total_attempts = 0
    total_games = 0

    win_count_by_length = collections.defaultdict(int)
    game_count_by_length = collections.defaultdict(int)

    # Collect all words from the validation loader
    all_words = []
    for batch in val_loader:
        batch_original_words = batch[-1]  # Adjust according to your batch structure
        all_words.extend(batch_original_words)

    # Randomly sample a set number of words for this epoch's validation
    selected_words = random.sample(all_words, min(max_games_per_epoch, len(all_words)))

    with torch.no_grad():
        for word in selected_words:
            won, final_word, attempts_used = simulate_game(
                model, 
                word, 
                char_to_idx, 
                idx_to_char, 
                char_frequency, 
                max_word_length, 
                device, 
                normalize=True, 
                max_attempts=6
            )
            total_wins += int(won)
            total_attempts += attempts_used
            total_games += 1

            word_length = len(word)
            win_count_by_length[word_length] += int(won)
            game_count_by_length[word_length] += 1

    win_rate = (total_wins / total_games) * 100 if total_games > 0 else 0
    average_attempts = total_attempts / total_games if total_games > 0 else 0
    win_rate_by_length = {length: (win_count_by_length[length] / game_count_by_length[length]) 
                          for length in game_count_by_length}

    return win_rate, average_attempts, win_rate_by_length


In [13]:
# optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
# num_epochs = 20

# avg_actual_penalty, avg_miss_penalty = train_one_epoch(model, data_loader, optimizer, \
#         device)

In [14]:
from torch.utils.data import DataLoader
# Training loop
from sklearn.model_selection import train_test_split


optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
num_epochs = 25

# Data Loaders
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# for epoch in range(num_epochs):
#     # Training step
#     avg_actual_penalty, avg_miss_penalty = train_one_epoch(model, train_loader, optimizer, \
#         device)
#     print(f"Epoch {epoch+1}: Training - Avg Actual Penalty: {avg_actual_penalty}, Avg Miss Penalty: {avg_miss_penalty}")

#     # Validation step
#     win_rate, average_attempts, win_rate_by_length = validate_one_epoch(model, \
#         val_loader, char_to_idx, idx_to_char, \
#     char_frequency, max_word_length, device)
#     print(f"Epoch {epoch+1}: Validation - Accuracy: {win_rate}%, win_rate_by_length: {average_attempts}, win_rate_by_length: {win_rate_by_length}")

#     print()

# win_rate, average_attempts, win_rate_by_length = validate_one_epoch(model, val_loader, \
#     char_to_idx, idx_to_char, char_frequency, max_word_length, device)

In [15]:
import optuna
from sklearn.model_selection import KFold
import torch
from torch.utils.data import DataLoader, Subset
from scr.model import RNN
from torch.optim.lr_scheduler import StepLR
from scr.utils import *
from scr.early_stopping import EarlyStopping

def objective(trial, dataset, static_config, num_epochs):
    dynamic_config = optuna_dynamic_hyperparameters(trial)
    config = {**static_config, **dynamic_config}
    # print(f"Trial {trial.number}: Configuration - {config}")

    kfold = KFold(n_splits=5, shuffle=True)
    aggregated_metrics = []

    best_objective_value = float('-inf')
    best_model_state = None

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        # print(f"Starting Fold {fold + 1}")
        train_loader, val_loader = get_data_loaders(dataset, train_idx, val_idx)
        model, optimizer = initialize_model(config)

        scheduler = StepLR(optimizer, step_size=trial.suggest_int("step_size", 5, 20), \
            gamma=trial.suggest_float("gamma", 0.1, 0.5))
        
        early_stopping = EarlyStopping(patience=10, delta=0.001)  # Early stopping instance


        for epoch in range(num_epochs):
            _, _ = train_one_epoch(model, train_loader, optimizer)
            scheduler.step()

            win_rate, average_attempts, win_rate_by_length = validate_one_epoch(model, \
                val_loader, char_to_idx, idx_to_char, char_frequency, max_word_length, device)

            aggregated_metrics.append((win_rate, average_attempts, win_rate_by_length))

            objective_value = calculate_objective_value(win_rate, average_attempts, win_rate_by_length)
            
            trial.report(objective_value, epoch)

            if objective_value > best_objective_value:
                best_objective_value = objective_value
                best_model_state = model.state_dict()  # Update best model state

            # Check early stopping
            if early_stopping(objective_value):
                break  # Stop the current fold if early stopping criteria are met

            # Pruning check
            if trial.should_prune():
                return process_aggregated_metrics(aggregated_metrics)

    # Save the best model at the end of the trial
    if best_model_state:
        model.load_state_dict(best_model_state)
        model.save_model()  # Save the model using your custom method

    final_objective_value = process_aggregated_metrics(aggregated_metrics)
    
    return final_objective_value


def calculate_objective_value(win_rate_percentage, avg_attempts_per_game, \
    win_rate_by_word_length):
    # Define weights for each metric
    weight_for_win_rate = 0.7  # Higher weight for win rate
    weight_for_average_attempts = 0.3  # Lower weight for average attempts

    # Normalize win rate (converting percentage to a range of 0 to 1)
    normalized_win_rate = win_rate_percentage / 100

    # Normalize average attempts (assuming max attempts per game is known)
    max_attempts_per_game = 6  # Example: maximum number of attempts allowed
    normalized_avg_attempts = avg_attempts_per_game / max_attempts_per_game

    # Normalize win rate by word length
    normalized_win_rate_by_length = normalize_win_rate_by_length(win_rate_by_word_length)

    # Combine metrics into a single objective value
    objective_value = (weight_for_win_rate * normalized_win_rate) - \
                      (weight_for_average_attempts * normalized_avg_attempts) + \
                      normalized_win_rate_by_length
    return objective_value

def normalize_win_rate_by_length(win_rate_by_length_dict):
    # Example logic for normalization
    max_possible_win_rate = 1  # Maximum win rate (100% as a decimal)
    normalized_scores = {length: win_rate / max_possible_win_rate for \
        length, win_rate in win_rate_by_length_dict.items()}
    # Average the normalized win rates across all word lengths
    average_normalized_score = sum(normalized_scores.values()) / len(normalized_scores)
    return average_normalized_score

def process_aggregated_metrics(aggregated_metrics):
    # Initialize sums for each metric
    sum_win_rate = 0
    sum_average_attempts = 0
    sum_win_rate_by_length = {}

    # Process each set of metrics
    for win_rate, average_attempts, win_rate_by_length in aggregated_metrics:
        sum_win_rate += win_rate
        sum_average_attempts += average_attempts
        for length, rate in win_rate_by_length.items():
            sum_win_rate_by_length[length] = sum_win_rate_by_length.get(length, 0) + rate

    # Calculate averages
    num_entries = len(aggregated_metrics)
    avg_win_rate = sum_win_rate / num_entries
    avg_average_attempts = sum_average_attempts / num_entries
    avg_win_rate_by_length = {length: rate / num_entries for length, rate in sum_win_rate_by_length.items()}

    # Now calculate the final objective value using these averages
    final_objective_value = calculate_objective_value(avg_win_rate, \
        avg_average_attempts, avg_win_rate_by_length)
    return final_objective_value


# Utility functions (for cleaner code)
def get_data_loaders(dataset, train_idx, val_idx):
    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, \
        shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_subset, batch_size=32, \
        shuffle=False, collate_fn=collate_fn)
    return train_loader, val_loader

def initialize_model(config):
    model = RNN(config)
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
    model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    return model, optimizer

# Utility functions
def optuna_dynamic_hyperparameters(trial):
    # Define and return dynamic hyperparameters based on the trial
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    hidden_dim = trial.suggest_categorical('hidden_dim', [128, 256, 512])
    embedding_dim = trial.suggest_categorical('embedding_dim', [50, 100, 150])
    output_mid_features = trial.suggest_categorical('output_mid_features', [50, 100, 200])
    miss_linear_dim = trial.suggest_categorical('miss_linear_dim', [50, 100, 150])
    dropout = trial.suggest_float('dropout', 0.2, 0.5)
    num_layers = trial.suggest_int('num_layers', 1, 3)

    return {
        'lr': lr,
        'hidden_dim': hidden_dim,
        'embedding_dim': embedding_dim,
        'output_mid_features': output_mid_features,
        'miss_linear_dim': miss_linear_dim,
        'dropout': dropout,
        'num_layers': num_layers
    }

In [16]:
from pathlib import Path

# Define your models directory path
models_dir = Path('models')

# Create the directory if it does not exist
models_dir.mkdir(parents=True, exist_ok=True)

# # Assuming missed_chars_tensor is a list of tensors for missed characters
# dataset = HangmanDataset(features, \
#     labels, missed_chars, original_words)

# Static configuration
static_config = {
    'rnn': 'LSTM',
    'vocab_size': 27,  # 26 English alphabets + 1 (e.g., for underscore)
    'use_embedding': True,  # Typically a design choice
    'input_feature_size': 5,  # Based on your feature engineering strategy
    'models': str(models_dir)
}

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define num_epochs for training in each fold
num_epochs = 20 # Adjust as needed
N_TRIAL = 25

# ====================================================== #
# Define the direction for the objective: 'maximize' or 'minimize'
direction = 'maximize'  # Adjust based on your objective function

study = optuna.create_study(direction=direction)
study.optimize(lambda trial: objective(trial, dataset, \
    static_config, num_epochs), n_trials=N_TRIAL)
# ====================================================== #

[I 2023-11-16 14:59:52,687] A new study created in memory with name: no-name-b5a017de-bc10-45f8-866b-b4c58de5b628
[I 2023-11-16 17:18:20,986] Trial 0 finished with value: -0.26674618565818786 and parameters: {'lr': 0.00017445130534251728, 'hidden_dim': 128, 'embedding_dim': 100, 'output_mid_features': 50, 'miss_linear_dim': 100, 'dropout': 0.23334078392375454, 'num_layers': 3, 'step_size': 7, 'gamma': 0.16042801409919455}. Best is trial 0 with value: -0.26674618565818786.
[I 2023-11-16 19:39:59,921] Trial 1 finished with value: -0.25860093296700803 and parameters: {'lr': 1.3683399679827584e-05, 'hidden_dim': 512, 'embedding_dim': 50, 'output_mid_features': 100, 'miss_linear_dim': 150, 'dropout': 0.32398449126972173, 'num_layers': 2, 'step_size': 12, 'gamma': 0.4037849211832595}. Best is trial 1 with value: -0.25860093296700803.
[I 2023-11-16 22:53:30,961] Trial 2 finished with value: -0.26295785259206866 and parameters: {'lr': 2.2110380911531494e-05, 'hidden_dim': 512, 'embedding_dim':

In [17]:
best_params = study.best_params
# Output the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"Value: {trial.value}")
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Best trial:
Value: -0.25200344544870795
Params: 
    lr: 1.0187940470581784e-05
    hidden_dim: 512
    embedding_dim: 50
    output_mid_features: 50
    miss_linear_dim: 100
    dropout: 0.30445129196460996
    num_layers: 1
    step_size: 9
    gamma: 0.33676963790615766


In [19]:
best_trial = study.best_trial
print(f"Best trial number: {best_trial.number}")

# Assuming model saving includes the trial number in the filename
best_model_filename = 'models/trial_1/LSTM.pth' # f"models/trial_{best_trial.number+1}/LSTM.pth"
best_model = RNN.load_model(RNN, best_model_filename, device)

Best trial number: 16


In [21]:
# Merge the two dictionaries to form a single configuration
config = {**static_config, **best_params}

import yaml
from pathlib import Path

# Assuming static_config and best_params are defined dictionaries
config = {**static_config, **best_params}

# Define the folder path using pathlib
folder_path = Path("models/trial_1")  # Replace with your actual folder path

# Ensure the folder exists
folder_path.mkdir(parents=True, exist_ok=True)

# Define the file path for the YAML file
file_path = folder_path / "config.yaml"

# Write the configuration to the YAML file
with open(file_path, 'w') as file:
    yaml.dump(config, file)


In [26]:
import torch
from scr.model import RNN  # Assuming RNN is your model class
from pathlib import Path

# Define the folder path and filenames
folder_path = Path("models/trial_1/")
model_filename = folder_path / "LSTM.pth"

# Load the saved file, which contains more than just the state dictionary
checkpoint = torch.load(model_filename, map_location=torch.device('cpu'))

# Extract the state dictionary for the model
model_state_dict = checkpoint['model_state_dict']

# Initialize the RNN model (ensure config is loaded or defined)
# Assuming you have the configuration dictionary for initializing RNN
config = checkpoint['config']  # Or load config from another source if needed
model = RNN(config)

# Load the state dictionary into the model
model.load_state_dict(model_state_dict)
model = model.to(torch.device('cpu'))
model.eval()  # Set to evaluation mode

# Now your model is loaded with the saved parameters and ready for inference or further training

RNN(
  (embedding): Embedding(28, 50)
  (linear1_out): Linear(in_features=1124, out_features=50, bias=True)
  (relu): ReLU()
  (linear2_out): Linear(in_features=50, out_features=27, bias=True)
  (miss_linear): Linear(in_features=27, out_features=100, bias=True)
  (rnn): LSTM(54, 512, num_layers=2, batch_first=True, dropout=0.3279967135638548, bidirectional=True)
)

In [ ]:
STOP

In [24]:
# Merge the two dictionaries to form a single configuration
config = {**static_config, **best_params}

# Initialize the RNN model with the combined configuration
model = RNN(config)

model = model.to(device)

# model.load_model(config['LSTM'], 2, 256, trial_number=10)

In [ ]:
STOP

##### Testing on unknown data

In [ ]:
%%capture

from scr.game import predict_next_character, simulate_game
from scr.model import RNN
import random
# random.seed(400)
# Your existing code for initializing the model, etc.

import random

def play_multiple_games(model, num_games, word_list, \
    char_to_idx, idx_to_char, char_frequency, max_word_length, device):
    game_results = []
    
    sampled_words = random.sample(word_list, num_games)  # Select unique words

    for random_word in sampled_words:
        with torch.no_grad():
            won, final_word, attempts_used = simulate_game(
                model, 
                random_word, 
                char_to_idx, 
                idx_to_char, 
                char_frequency, 
                max_word_length, 
                device, 
                normalize=True, 
                max_attempts=6
            )
        game_results.append((won, final_word, attempts_used))

    return game_results

# Example usage
num_games = 10**3
results = play_multiple_games(model, num_games, \
    word_list, char_to_idx, idx_to_char, \
        char_frequency, max_word_length, device)

# Analyzing results
total_wins = sum(result[0] for result in results)
win_rate = (total_wins / num_games) * 100
